In [1]:
import pandas as pd;
import numpy as np;
import lightgbm as lgb
#from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK
from hyperopt import hp

MAX_EVALS = 500
N_FOLDS = 10



Hyperopt

Hyperopt is one of several automated hyperparameter tuning libraries using Bayesian optimization. These libraries differ in the algorithm used to both construct the surrogate (probability model) of the objective function and choose the next hyperparameters to evaluate in the objective function. Hyperopt uses the Tree Parzen Estimator (TPE). Other Python libraries include Spearmint, which uses a Gaussian process for the surrogate, and SMAC, which uses a random forest regression.

Hyperopt has a simple syntax for structuring an optimization problem which extends beyond hyperparameter tuning to any problem that involves minimizing a function. Moreover, the structure of a Bayesian Optimization problem is similar across the libraries, with the major differences coming in the syntax (and in the algorithms behind the scenes that we do not have to deal with).


In [2]:
# import Dataset to play with it
train= pd.read_csv("/home/altieris/datascience/data/santander-customer-transaction-prediction/train.csv")

In [3]:
cols=["target","ID_code"]

train_x = train.drop(cols,axis=1)
train_y = train["target"]


In [ ]:
# %%time
# idx = features = train_x.columns.values[0:123]
# for df in [train_x]:
#     df['sum'] = df[idx].sum(axis=1)  
#     df['min'] = df[idx].min(axis=1)
#     df['max'] = df[idx].max(axis=1)
#     df['mean'] = df[idx].mean(axis=1)
#     df['std'] = df[idx].std(axis=1)
#     df['skew'] = df[idx].skew(axis=1)
#     df['kurt'] = df[idx].kurtosis(axis=1)
#     df['med'] = df[idx].median(axis=1)

In [4]:
train_x.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
train_set = lgb.Dataset(train_x, label=train_y)

In [6]:
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
 
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['min_data_in_leaf','bagging_freq','num_leaves']: #, 'subsample_for_bin', 'min_child_samples'
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 1000000, nfold = n_folds, early_stopping_rounds = 3500, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# param = {
#     'bagging_freq': 5,          
#     'bagging_fraction': 0.32,   
#     'boost_from_average':'false',  
    
#     'boost': 'gbdt',
    
#     'feature_fraction': 0.035,   
#     'learning_rate': 0.008,     
#     'max_depth': -1,                
#     'metric':'auc',
    
#     'min_data_in_leaf': 80,     
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_leaves': 13,           
#     'num_threads': 8,
#     'tree_learner': 'serial',   
#     'objective': 'binary',      
#     'verbosity': 1
# }

In [7]:
# Define the search space
space = {
    'boosting_type': 'gbdt',
    'metric':'auc',
    'num_threads': 8,
    'tree_learner': 'serial',   
    'objective': 'binary', 
    
    'bagging_freq': hp.quniform('bagging_freq', 1, 5, 1),          
    'bagging_fraction': hp.uniform('bagging_fraction', 0.0, 0.4),   
    'boost_from_average':'false',  
    'feature_fraction': hp.uniform('feature_fraction', 0.02, 0.05),   
    'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.1)),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 50, 100, 5), 
    'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 1, 10, 1), 
    'num_leaves': hp.quniform('num_leaves', 7, 15, 1) 

}

In [8]:
from hyperopt import tpe

# optimization algorithm
tpe_algorithm = tpe.suggest

In [9]:
from hyperopt import Trials

# Keep track of results
bayes_trials = Trials()

In [10]:
# File to save first results
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

In [11]:
from hyperopt import fmin

In [ ]:
%%capture

# Global variable
global  ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

In [ ]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results[:2]